Hay que hacer bonito esto pero es pa poner la direccion de la api 


In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup

In [44]:
lista_paises = [["japon", (1998, 2006), "WC"], ["corea-del-sur", (1998, 2006), "WC"], 
["alemania", (2002, 2010), "WC"], ["sudafrica", (2006, 2014), "WC"], ["brasil", (2010, 2018), "WC"], ["rusia", (2014, 2020), "WC"], 
["grecia", (2000, 2008), "OL"], ["china", (2004, 2012), "OL"], ["uk", (2008, 2016), "OL"], ["brasil", (2012, 2020), "OL"]]
l_ol = []
l_wc = []

In [47]:
for j in lista_paises:
    pais = j[0]
    anos = j[1]
    tipo = j[2]
    url = f"https://datosmacro.expansion.com/comercio/turismo-internacional/{pais}"
    page = rq.get(url)
    sopa = BeautifulSoup(page.content, "html.parser")
    tabla = sopa.find("table")
    valores = tabla.find_all('td')

    fechas = []
    gente = []
    for i in range(len(valores)):
        if i%2 == 0:
            fechas.append(valores[i].text)
        else:
            gente.append(valores[i].text)
    
    df = pd.DataFrame(columns=['Fecha', 'Turistas'])
    df['Fecha'] = fechas
    df['Turistas'] = gente
    df = df.sort_values('Fecha').reset_index(drop=True)
    df['Fecha'] = df['Fecha'].astype('int64')
    df['Turistas'] = df['Turistas'].apply(lambda x: x.replace('.', ''))
    df['Turistas'] = df['Turistas'].astype('int64')

    porcentaje = []
    for i in df["Turistas"]:
        if len(porcentaje) == 0:
            porcentaje.append(0)
            ultimo = i
        else:
            porc = (i*100)/ultimo
            porc -= 100
            porcentaje.append(porc)
            ultimo = i
    
    df['crecimiento'] = porcentaje
    df = df[df["Fecha"] <= anos[1]]
    df = df[df["Fecha"] >= anos[0]]
    df["Pais"] = pais
    df["Evento"] = tipo
    df.reset_index(drop=True, inplace=True)

    if tipo == "WC":
        l_wc.append(df)

    else:
        l_ol.append(df)
    

Ver crecimiento mundial de pib, promedio y calcular el promedio de cada pais y comparar

Hipótesis:   
1. La organización de un mundial hace que en el tiempo cercano al evento (los 2 años antes) se incrementa el gasto en el turismo, ya que se espera que el mundial fomente el turismo en el país.   
2. El mundial genera una mayor llegada de turistas en el año y en el próximo lustro.   
3. La mayor llegada de turistas genera más ingresos por turismo.   
4. Esto al final no influye mucho en el pib (no genera más de medio punto de cambio en el per capita)

Organizar:
Calcular diferencia porcentual por pais del pib, ingreso neto, ingreso personas
Despues de calcular eso:
3 graficos pequeños con lo calculado, grafico por pais (pairplot)
Final:
3 graficos finales con 4 antes por pais (color por wc y ol) y 4 despues con lo mismo y asi hacer difecrencia por pais
